In [12]:
import pandas as pd
import numpy as np

weeks = ['170506','170513','170520', '170527','170603']
list_of_df =[pd.DataFrame()]*5;
for i in range(5):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + weeks[i] + ".txt"
    list_of_df[i] = pd.read_csv(url)
df = pd.concat(list_of_df)
df.columns = [column.strip() for column in df.columns]

In [13]:
df = df.reset_index()
df = df.drop('index',1)
df.tail()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
981042,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,05:00:00,REGULAR,5554,299
981043,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,09:00:00,REGULAR,5554,299
981044,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,13:00:00,REGULAR,5554,299
981045,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,17:00:00,REGULAR,5554,299
981046,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/02/2017,21:00:00,REGULAR,5554,299


In [15]:
#add a DATETIME column from DATE and TIME
df['DATETIME'] = [pd.to_datetime(i+j, format = '%m/%d/%Y%H:%M:%S') for i, j in zip(df.DATE, df.TIME)]

In [16]:
#find duplicate entries
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE", "DATETIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE,DATETIME,ENTRIES
871022,R504,R276,00-00-01,VERNON-JACKSON,05/26/2017,2017-05-26 20:00:00,2
135038,G001,R151,00-00-00,CONEY IS-STILLW,05/09/2017,2017-05-09 05:00:00,2
0,A002,R051,02-00-00,59 ST,04/29/2017,2017-04-29 00:00:00,1
654035,R145,R032,00-00-03,TIMES SQ-42 ST,05/13/2017,2017-05-13 12:00:00,1
654024,R145,R032,00-00-03,TIMES SQ-42 ST,05/11/2017,2017-05-11 16:00:00,1


In [17]:
#investigate the two duplicate rows in ENTRIES and chose to delete them
#did not find any duplicates in EXITS column

mask = ((df["C/A"] == "R504") & 
(df["UNIT"] == "R276") & 
(df["SCP"] == "00-00-01") & 
(df["STATION"] == "VERNON-JACKSON") &
(df["DATE"] == "05/26/2017"))
df[mask]
df.drop(df.index[763079], inplace = True)
df[mask]

mask = ((df["C/A"] == "G001") & 
(df["UNIT"] == "R151") & 
(df["SCP"] == "00-00-00") & 
(df["STATION"] == "CONEY IS-STILLW") &
(df["DATE"] == "05/09/2017"))
df[mask]
df.drop(df.index[224251], inplace = True)
df[mask]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
224250,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,01:00:00,REGULAR,199982,337867,2017-05-09 01:00:00
224252,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,05:00:00,RECOVR AUD,200021,337947,2017-05-09 05:00:00
224253,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,09:00:00,REGULAR,200363,338280,2017-05-09 09:00:00
224254,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,13:00:00,REGULAR,200546,338821,2017-05-09 13:00:00
224255,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,17:00:00,REGULAR,200860,339305,2017-05-09 17:00:00
224256,G001,R151,00-00-00,CONEY IS-STILLW,DFNQ,BMT,05/09/2017,21:00:00,REGULAR,201098,339691,2017-05-09 21:00:00


In [18]:
df = df.reset_index()

In [19]:
del df['DESC']
df['ENTRIESDOWN'] = df.groupby(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION']).ENTRIES.shift()
df['EXITSDOWN'] = df.groupby(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION']).EXITS.shift()
df.dropna(inplace = True)
df['ENTRIESDOWN'] = [int(j) for j in df['ENTRIESDOWN']]
df['EXITSDOWN'] = [int(j) for j in df['EXITSDOWN']]
df.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,ENTRIESDOWN,EXITSDOWN
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,6157777,2085319,2017-04-29 04:00:00,6157740,2085315
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,6157810,2085353,2017-04-29 08:00:00,6157777,2085319
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,6157963,2085453,2017-04-29 12:00:00,6157810,2085353
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,6158212,2085529,2017-04-29 16:00:00,6157963,2085453
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,20:00:00,6158521,2085589,2017-04-29 20:00:00,6158212,2085529


In [20]:
#geting the count values from the cumulative entries and exits

df['ENTRIES'] = [i - int(j) for i,j in zip(df['ENTRIES'], df['ENTRIESDOWN'])]
df['EXITS'] = [i - int(j) for i,j in zip(df['EXITS'], df['EXITSDOWN'])]


In [21]:
#deleting rows where ENTRIES or EXITS are negative or greater than 144,000
#assuming 144,000 is the maximum amount of people who would go through a turnstile in a 4 hour period (10 people/second)

mask = ((df["ENTRIES"] > -1) & 
(df["EXITS"] > -1))
df = df[mask]
mask = ((df["ENTRIES"]<144000)&
       (df["EXITS"]<144000))
df = df[mask]

In [22]:
del df['ENTRIESDOWN']
del df['EXITSDOWN']

In [23]:
df.to_csv('Project1BestData.csv')